#stuff

In [ ]:
!rm -rf /content/...

#Vietnamese Word Segmentation (vncorenlp and UITws)

vncorenlp

In [ ]:
!pip3 install py_vncorenlp

In [ ]:
import py_vncorenlp
import os
cwd = os.getcwd()
py_vncorenlp.download_model(save_dir=cwd)
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir=cwd)

In [ ]:
# for single text
segmented_text = rdrsegmenter.word_segment(text)

In [ ]:
# for list of sentences
segmented_texts = [rdrsegmenter.word_segment(comment)[0] for comment in dataset['train']['sentence']]

UITws

In [ ]:
!pip3 install --user gitdir
!/root/.local/bin/gitdir https://github.com/chaoswithinyou/nlp_tools_ndsl/tree/main/word_segmentation

In [ ]:
from word_segmentation.UITws import word_segment

In [ ]:
# for single text
word_segment(t, single_text=True)

In [ ]:
# for list of sentence
word_segment(t)

#Vietnamese Word2vec (PhoW2V)

In [ ]:
import os
cwd = os.getcwd()

In [ ]:
import zipfile
with zipfile.ZipFile(cwd+'/word2vec_vi_words_100dims.zip', 'r') as zip_ref:
    zip_ref.extractall(cwd)

In [ ]:
# download w2v_vi_100dims
!gdown --id 1--bdQosASIlc3O4xN5IZ5vLfZ_yEdfBZ

In [ ]:
# download w2v_vi_300dims
!gdown --id 1rA7rGOEuhzfTzEiSiH-MiqWhQeR_ETTi

In [ ]:
PhoW2V_300dims = pickle.load(open(cwd+'/vietnamese_PhoW2V_300dims.pickle','rb'))

In [ ]:
PhoW2V_100dims = pickle.load(open(cwd+'/vietnamese_PhoW2V_00dims.pickle','rb'))

In [ ]:
#pickle.dump(net,open('/content/drive/MyDrive/NLP/imdb_sentiment.p','wb'))

In [ ]:
#PhoW2V = [words, word2idx, vectors]
vietnamese_w2v = {w: PhoW2V[2][PhoW2V[1][w]] for w in PhoW2V[0]}

In [ ]:
def word2ids(sen_list, maxlen, vector_size):
    x = torch.zeros((len(sen_list),1,maxlen))
    for i in tqdm(range(len(sen_list))):
      doc = tokenize.word_tokenize(sen_list[i])
      fil_doc_w2index = []
      for word in doc:
        try:
          fil_doc_w2index.append(PhoW2V[1][word])
        except Exception:
          pass
      if len(fil_doc_w2index)<=maxlen:
        x[i] = torch.cat((torch.LongTensor(fil_doc_w2index),torch.zeros(maxlen-len(fil_doc_w2index)))).unsqueeze(0)
      else:
        x[i] = torch.LongTensor(fil_doc_w2index[-maxlen-1:-1]).unsqueeze(0)
    return x

In [ ]:
input_ids = word2ids(sen_list = segmented_texts, maxlen = 50, vector_size = 100)

In [ ]:
import torch
import torch.nn as nn

def create_emb_layer_from_vector_mat(weights_matrix, non_trainable=False):
    weights_matrix = torch.Tensor(weights_matrix)
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

In [ ]:
def __init__(self):
        super(Net, self).__init__()
        self.embedding, num_embeddings, vector_size = create_emb_layer(vectors, non_trainable=True)
def forward(self, x):
        x = self.embedding(x)

#PhoBart

In [ ]:
!pip install transformers
from transformers import AutoModel, AutoTokenizer

In [ ]:
word_tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word")
bartpho_word = AutoModel.from_pretrained("vinai/bartpho-word")

In [ ]:
# segmentation_text are list of sentences(output of word segmentation), input_ids 
input_ids = word_tokenizer(segmented_texts, return_tensors='pt', padding=True)['input_ids']

In [ ]:
vector_size = 1024
bartpho_word = bartpho_word.to(device)
def forward(self, x):
    with torch.no_grad():
        x = bartpho_word(x).last_hidden_state.unsqueeze(1)

#List of sentences to 2d list of ids

In [ ]:
from tqdm import tqdm
from nltk import tokenize

def word2ids(sen_list, word2idx, maxlen, vector_size):
    x = torch.zeros((len(sen_list),1,maxlen))
    for i in tqdm(range(len(sen_list))):
      doc = tokenize.word_tokenize(sen_list[i])
      fil_doc_w2index = []
      for word in doc:
        try:
          fil_doc_w2index.append(word2idx[word])
        except Exception:
          pass
      if len(fil_doc_w2index)<=maxlen:
        x[i] = torch.cat((torch.LongTensor(fil_doc_w2index),torch.zeros(maxlen-len(fil_doc_w2index)))).unsqueeze(0)
      else:
        x[i] = torch.LongTensor(fil_doc_w2index[-maxlen-1:-1]).unsqueeze(0)
    return x

#MyDataset class

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
class MyDataset(Dataset):
  def __init__(self, x, y):
    self.data = x
    self.labels = y

  def __len__(self):
    return len(self.labels)
  
  def __getitem__(self, index):
    return self.data[index], self.labels[index]

In [ ]:
train_data = MyDataset(input_ids, train_dataset['sentiment'])
test_data = MyDataset(input_ids2, test_dataset['sentiment'])

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True, num_workers=2)

#Accuracy class and usage

In [ ]:
class Accuracy:
    """A class to keep track of the accuracy while training"""
    def __init__(self):
        self.correct = 0
        self.total = 0
        
    def reset(self):
        """Resets the internal state"""
        self.correct = 0
        self.total = 0
        
    def update(self, outputs, labels):
        """
        Updates the internal state to later compute the overall accuracy
        
        output: the output of the network for a batch
        labels: the target labels
        """
        _, predicted = torch.max(outputs.data, 1) # predicted now contains the predicted class index/label
        
        self.total += labels.size(0)
        self.correct += (predicted == labels).sum().item() # .item() gets the number, not the tensor
        #self.correct += ((outputs.data > 0.5) == labels).sum().item()

    def compute(self):
        return self.correct/self.total

In [ ]:
def test_accu():
    net.eval()
    test_accuracy = Accuracy()
    test_accuracy.reset()        
    with torch.no_grad():
        for test_data in test_loader:
            # get the data points
            test_inputs, test_labels = test_data
            test_inputs, test_labels = test_inputs.to(device).long(), test_labels.to(device)
            # forward the data through the network
            test_outputs = net(test_inputs)
            
            test_accuracy.update(test_outputs, test_labels)
            
    print("\nTesting Accuracy: {:.2f}%".format(100 * test_accuracy.compute()))

#Model ini and training

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
bartpho_word = bartpho_word.to(device)
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1,
                                out_channels = 100,
                                kernel_size = (2,vector_size))
        self.conv2 = nn.Conv2d(in_channels = 1,
                                out_channels = 100,
                                kernel_size = (3,vector_size))
        self.conv3 = nn.Conv2d(in_channels = 1,
                                out_channels = 100,
                                kernel_size = (4,vector_size))
        self.conv4 = nn.Conv2d(in_channels = 1,
                                out_channels = 100,
                                kernel_size = (5,vector_size))
        self.fc1 = nn.Linear(400, 3)
        self.dropout = nn.Dropout(0.30)
        self.dropout1 = nn.Dropout(0.30)
        self.dropout2 = nn.Dropout(0.30)
        self.dropout3 = nn.Dropout(0.30)
        self.dropout4 = nn.Dropout(0.30)
        self.dropout5 = nn.Dropout(0.30)
    def forward(self, x):
        with torch.no_grad():
            x = bartpho_word(x).last_hidden_state.unsqueeze(1)
        x = self.dropout(x)
        x1 = F.relu(self.conv1(x))
        x1 = self.dropout1(F.max_pool2d(x1, (x1.size()[-2],1)))
        x2 = F.relu(self.conv2(x))
        x2 = self.dropout2(F.max_pool2d(x2, (x2.size()[-2],1)))
        x3 = F.relu(self.conv3(x))
        x3 = self.dropout3(F.max_pool2d(x3, (x3.size()[-2],1)))
        x4 = F.relu(self.conv4(x))
        x4 = self.dropout4(F.max_pool2d(x4, (x4.size()[-2],1)))
        x = torch.flatten(torch.cat((x1,x2,x3,x4),-3),start_dim=-3)
        x = self.fc1(self.dropout5(x))
        return x

net = Net()
net = net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
#optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.5)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.num_layers =  num_layers
        self.hidden_size = hidden_size

        #self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # x -> (batch_size, sequence_length, input_size) because batch_size = true
        self.fc1 = nn.Linear(hidden_size, 32)
        self.fc2 = nn.Linear(32, num_classes)
        #self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout1 = nn.Dropout(0.30)
        self.dropout2 = nn.Dropout(0.30)

    def forward(self, x):
        # initial hidden state size is always (num_layer, batch_size, hidden_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        #out, _ = self.rnn(x, h0)
        #out, _ = self.gru(x, h0)
        out, _ = self.lstm(x, (h0,c0))
        
        # out -> (batch_size, sequence_length, hidden_size) because batch_size = true
        out = out[:, -1, :] # only the last time step

        out = F.relu(self.fc1(self.dropout1(out)))
        out = self.fc2(self.dropout2(out))
        #out = self.fc(self.dropout1(out))
        return out

net = LSTM(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.5)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class CNNLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(CNNLSTM, self).__init__()

        self.weights = ResNet50_Weights.DEFAULT
        self.preprocess = self.weights.transforms()
        self.pretrained_cnn = resnet50(weights=self.weights)

        #self.resnet.fc = nn.Sequential(nn.Linear(self.resnet.fc.in_features, 300))
        self.pretrained_cnn.fc = nn.Linear(self.pretrained_cnn.fc.in_features, input_size)

        self.num_layers =  num_layers
        self.hidden_size = hidden_size


        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # x -> (batch_size, sequence_length, input_size) because batch_size = true
        self.fc1 = nn.Linear(hidden_size, 32)
        self.fc2 = nn.Linear(32, num_classes)
        self.dropout = nn.Dropout(0.40)

    def forward(self, x):
        hidden = None
        for i in range(x.size(1)):
            with torch.no_grad():
                out = self.pretrained_cnn(self.preprocess(x[:,i,:,:,:]))
            out, hidden = self.lstm(out.unsqueeze(1), hidden)
        
        # out -> (batch_size, sequence_length, hidden_size) because batch_size = true
        out = out[:, -1, :] # only the last time step

        out = F.relu(self.fc1(out))
        out = self.fc2(self.dropout(out))

        return out

net = CNNLSTM(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.5)

In [ ]:
#net = net.float()
net.train()
live_accuracy = Accuracy()

for epoch in range(100):  # loop over the dataset multiple times
    print("\nStarting epoch {}".format(epoch+1))

    live_accuracy.reset()
    total = 0
    running_loss = 0.0

    # to make a beautiful progress bar
    loader = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, data in loader:
        # get the data points
        inputs, labels = data
        inputs, labels = inputs.to(device).long(), labels.to(device)
        # zero the parameter gradients (else, they are accumulated)
        optimizer.zero_grad()

        # forward the data through the network
        outputs = net(inputs)
        # calculate the loss given the output of the network and the target labels
        loss = criterion(outputs, labels)
        # calculate the gradients of the network w.r.t. its parameters
        loss.backward()
        # Let the optimiser take an optimization step using the calculated gradients
        optimizer.step()
        
        running_loss += loss
        total += outputs.size(0)

        live_accuracy.update(outputs, labels)
        loader.set_description("loss: {:.5f}|acc: {:.2f}%".format(running_loss/total,100 * live_accuracy.compute()))
    test_accu()
    net.train()

print('Finished Training')

#Visualization

In [ ]:
# in test mode, we should set the 'learning_phase' flag to 0 (we don't want to use dropout)
get_doc_embedding = copy.deepcopy(net)
get_doc_embedding.fc1 = torch.nn.Identity()
get_doc_embedding.eval()

num_sen = 1000
print('plotting embeddings of first',num_sen,'documents')

doc_emb = get_doc_embedding(test_data.data[:num_sen].long()).detach().numpy()

my_pca = PCA(n_components=10)
my_tsne = TSNE(n_components=2,perplexity=30)
doc_emb_pca = my_pca.fit_transform(doc_emb)
doc_emb_tsne = my_tsne.fit_transform(doc_emb_pca) 
#doc_emb_tsne = my_tsne.fit_transform(doc_emb)

labels_plt = test_data.labels[:num_sen]
my_colors = ['red','yellow','green']

fig, ax = plt.subplots()

for label in list(set(labels_plt)):
    idxs = [idx for idx,elt in enumerate(labels_plt) if elt==label]
    ax.scatter(doc_emb_tsne[idxs,0], 
               doc_emb_tsne[idxs,1], 
               c = my_colors[label],
               label=str(label),
               alpha=0.7,
               s=10)

ax.legend(scatterpoints=1)
fig.suptitle('t-SNE visualization of CNN-based doc embeddings \n (first 1000 docs from test set)',fontsize=10)
fig.set_size_inches(6,4)
#fig.savefig(path_to_plot + 'doc_embeddings_init.pdf',bbox_inches='tight')
fig.show()

In [1]:
!pip install -U transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 163 kB 57.2 MB/s 
     |████████████████████████████████| 6.6 MB 37.6 MB/s 


In [2]:
import torch
from transformers import AutoModel, AutoTokenizer, PhobertTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
sentence = 'Chúng_tôi là những nghiên_cứu_viên .'  

input_ids = torch.tensor([tokenizer.encode(sentence)])

with torch.no_grad():
    features = phobert(input_ids)  # Models outputs are now tuples

## With TensorFlow 2.0+:
# from transformers import TFAutoModel
# phobert = TFAutoModel.from_pretrained("vinai/phobert-base")

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/895k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
a = tokenizer.get_vocab()

In [ ]:
b = {v: k for k, v in a.items()}

In [5]:
a['Nắng']

15792

In [ ]:
b[63997]

'ັ@@'